Libraries


In [26]:
import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from datetime import datetime, date 

#from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
import tensorflow as tf

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 14})
import seaborn as sns

from IPython.display import display_html # To visualize dataframes side by side
import os
N_EPOCHS = 32
SEED = 42

# Fix seed for reproducible results
os.environ['PYTHONHASHSEED']=str(SEED)
seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


Read Data

In [27]:
#import pandas as pd
data=pd.read_csv("data.CSV" )
data['Date'] = pd.to_datetime(data['Date'])
dataset=data.set_index(data['Date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('Date', 1)
#dataset.head(2)
#data

<ipython-input-27-f03babacb84e>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dataset=data.set_index(data['Date'], drop=False, append=False, inplace=False, verify_integrity=False).drop('Date', 1)


In [28]:
def split_sequences(features, targets, n_steps_in, n_steps_out, n_sliding_steps, window_type):
    """
    Edited from 
    https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
    
    Args:
    * features: univariate or multivariate input sequences
    * targets: univariate or multivariate output sequences
    * n_steps_in: length of input sequence for sliding window. 
    * n_steps_out: length of output sequence
    * n_sliding_steps: window step size
    * window_type: 'sliding' or 'expanding'
    """
    X, y = list(), list()
    for i in range(0, len(features), n_sliding_steps):

        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequences
        if out_end_ix > len(features):
            break
            
        # gather input and output parts of the pattern
        if window_type == 'sliding': # Sliding window
            seq_x, seq_y = features[i:end_ix, :], targets[end_ix:out_end_ix, :]
        else: # expanding window
            seq_x, seq_y = features[0:end_ix, :], targets[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [31]:
input_seq_length = 24 
output_seq_length = 1 #prediction horizon

input_cols = ['Date', 'Load']
output_cols = input_cols

ts_df=data

X_all, y_all =split_sequences(ts_df[input_cols].values, 
                       ts_df[output_cols].values, 
                       n_steps_in = input_seq_length, 
                       n_steps_out = output_seq_length, 
                       n_sliding_steps = 5, 
                       window_type='sliding')

n_samples = X_all.shape[0]
print(f"Total of {n_samples} samples created.")

Total of 225 samples created.


In [17]:
# Data for training/validation
X = X_all[:(n_samples -output_seq_length)]
y = y_all[:(n_samples - output_seq_length)]

# Data for testing
X_test = X_all[(n_samples - output_seq_length):]
y_test = y_all[(n_samples - output_seq_length):]

n_features = X_all.shape[2] - 1 # column timestep is not a feature


# Split training and validation data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

# Covert to float and reshape
index_X_train = X_train[:, :, 0]
index_y_train = y_train[:, :, 0]
index_X_val = X_val[:, :, 0]
index_y_val = y_val[:, :, 0]
index_X_test = X_test[:, :, 0]
index_y_test = y_test[:, :, 0]

X_train = X_train[:, :, 1].astype(float).reshape((X_train.shape[0], input_seq_length, n_features))
y_train = y_train[:, :, 1].astype(float)

X_val = X_val[:, :, 1].astype(float).reshape((X_val.shape[0], input_seq_length, n_features))
y_val = y_val[:, :, 1].astype(float)

X_test = X_test[:, :, 1].astype(float).reshape((X_test.shape[0], input_seq_length, n_features))
y_test = y_test[:, :, 1].astype(float)

print(X_train.shape,X_val.shape,y_train.shape,y_val.shape )

(179, 24, 1) (45, 24, 1) (179, 1) (45, 1)
